In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from functools import reduce
from tabulate import tabulate
pd.set_option('display.max_columns', None)

In [2]:
filepath_books = "/Users/ortalhadad/Desktop/Sapienza/Year 1/Semester A/ADM/HW2/lighter_books.json"

> ## RQ3- Report1
>
> We realize that to construct a new data frame with year as primary key and the required information as attributes
> will take a very long running time if we have to run over the whole dataset for each year.   
> Therefore we decided to create a dictionary that contains year as key and the required attributes as value.   
> That way our function only take the values from the dictionary.  
> To create the dictionary we run over the data base only once.
> Before we started working on the question we realized that we needed to organize all the column types,   
> and especially the type of 'publication_date' and 'num_pages'.  
> We had to handel with:  
> - missing dates- with empty value in 'publication_date'  
> - dates that are not valid- like year '3001'  
> - dates that are not complete- as '2001-01'


In [3]:
## First we create a dictionary that contains year as key and the required attributes as value. 
def num_pages_fix(num):
    if num <= 0:
        return None
    else:
        return num 

def year_fix(year):
    if year > 2023:
        return 'Invalid'
    else:
        return year
    
chunks = pd.read_json(filepath_books, lines = True, chunksize = 10000)
result = {}
for chunk in chunks:
        df = chunk[['title', 'publication_date', 'num_pages']].copy()
        df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')
        df['num_pages'] = pd.to_numeric(df['num_pages'], errors='coerce')
        df['num_pages'] = df['num_pages'].map(num_pages_fix)
        df_h = df[df['publication_date'].notnull() & df['num_pages'].notnull()]
        
        df_h['year'] = df_h['publication_date'].dt.year
        df_h['year'] = df_h['year'].map(year_fix)
        df_h['month'] = df_h['publication_date'].dt.month
        grouped = df_h.groupby('year')
        for year, group in grouped:
            total_books = len(group)
            
            total_pages = group['num_pages'].sum()
            
            monthly_counts = group.groupby('month').size()
            most_prolific_month = monthly_counts.idxmax()
        
            longest_book = group[group['num_pages'] == group['num_pages'].max()]
            longest_book_num_pages = group['num_pages'].max()
            
            if year in result:
                result[year]['total_books'] += total_books
                result[year]['total_pages'] += total_pages
                
                merged_counts = monthly_counts.add(result[year]['monthly_counts'], fill_value=0)
                most_prolific_month = merged_counts.idxmax()
                result[year]['most_prolific_month'] = most_prolific_month
                result[year]['monthly_counts'] = merged_counts

                if result[year]['longest_book_num_pages'] < longest_book_num_pages:
                    result[year]['longest_book'] = longest_book 
                    result[year]['longest_book_num_pages'] = longest_book_num_pages
            else:
                result[year] = {'total_books': total_books, 'total_pages': total_pages, 'most_prolific_month': most_prolific_month, 'monthly_counts': monthly_counts, 'longest_book': longest_book, 'longest_book_num_pages': longest_book_num_pages}
print(result)

/var/folders/c2/t01w5t9j4ks01l3tm6pfc7rc0000gn/T/ipykernel_6731/3953507472.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h['year'] = df_h['publication_date'].dt.year
/var/folders/c2/t01w5t9j4ks01l3tm6pfc7rc0000gn/T/ipykernel_6731/3953507472.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h['year'] = df_h['year'].map(year_fix)
/var/folders/c2/t01w5t9j4ks01l3tm6pfc7rc0000gn/T/ipykernel_6731/3953507472.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

{1888: {'total_books': 150, 'total_pages': 43941.0, 'most_prolific_month': 1, 'monthly_counts': month
1     131.0
4      11.0
5       2.0
8       1.0
9       2.0
10      2.0
12      1.0
dtype: float64, 'longest_book':                                                      title publication_date  \
6447435  The Great Cryptogram: Francis Bacon's Cipher i...       1888-01-01   

         num_pages  year  month  
6447435      998.0  1888      1  , 'longest_book_num_pages': 998.0}, 1899: {'total_books': 333, 'total_pages': 113371.0, 'most_prolific_month': 1, 'monthly_counts': month
1     279.0
2       2.0
4       7.0
6       1.0
7       1.0
8       1.0
9       1.0
12     41.0
dtype: float64, 'longest_book':                                                      title publication_date  \
4496325  El libro de las mil y una noches: Edición comp...       1899-01-01   

         num_pages  year  month  
4496325     3613.0  1899      1  , 'longest_book_num_pages': 3613.0}, 1900: {'total_books': 2285,

In [4]:
## RQ3- Historical look at the dataset!

def year_historic(year: int):
    return result[year]['total_books'], result[year]['total_pages'], result[year]['most_prolific_month'], result[year]['longest_book']


In [18]:
## Get a list of all relevant publication years in the data frame
def year_fix(year):
    if year > 2023:
        return None
    else:
        return year

chunks = pd.read_json(filepath_books, lines = True, chunksize = 10000)
years = []
for chunk in chunks:
    df_historic = chunk[['publication_date', 'num_pages']].copy()
    df_historic['publication_date'] = pd.to_datetime(df_historic['publication_date'], errors='coerce')
    df_historic['year'] = df_historic['publication_date'].dt.year
    df_historic['year'] = df_historic['year'].map(year_fix)
    df_historic['num_pages'] = pd.to_numeric(df_historic['num_pages'], errors='coerce')
    df_historic['num_pages'] = df_historic['num_pages'].map(num_pages_fix)
    df_historic = df_historic[df_historic['year'].notnull() & df_historic['num_pages'].notnull()]
    years.extend(df_historic['year'].unique())
print(set(years))

{1678.0, 1679.0, 1680.0, 1681.0, 1682.0, 1684.0, 1686.0, 1688.0, 1689.0, 1691.0, 1692.0, 1694.0, 1696.0, 1697.0, 1698.0, 1700.0, 1701.0, 1703.0, 1705.0, 1710.0, 1711.0, 1713.0, 1714.0, 1715.0, 1717.0, 1719.0, 1720.0, 1722.0, 1724.0, 1725.0, 1726.0, 1728.0, 1729.0, 1731.0, 1732.0, 1733.0, 1734.0, 1736.0, 1737.0, 1740.0, 1741.0, 1742.0, 1746.0, 1747.0, 1748.0, 1749.0, 1750.0, 1751.0, 1752.0, 1753.0, 1755.0, 1756.0, 1757.0, 1758.0, 1759.0, 1760.0, 1762.0, 1765.0, 1766.0, 1767.0, 1768.0, 1769.0, 1771.0, 1773.0, 1774.0, 1775.0, 1776.0, 1778.0, 1779.0, 1780.0, 1781.0, 1782.0, 1783.0, 1784.0, 1785.0, 1786.0, 1787.0, 1788.0, 1789.0, 1790.0, 1791.0, 1792.0, 1793.0, 1794.0, 1795.0, 1796.0, 1797.0, 1798.0, 1799.0, 1800.0, 1801.0, 1802.0, 1803.0, 1804.0, 1805.0, 1806.0, 1807.0, 1808.0, 1809.0, 1810.0, 1811.0, 1812.0, 1813.0, 1814.0, 1815.0, 1816.0, 1817.0, 1818.0, 1819.0, 1820.0, 1821.0, 1822.0, 1823.0, 1824.0, 1825.0, 1826.0, 1827.0, 1828.0, 1829.0, 1830.0, 1831.0, 1832.0, 1833.0, 1834.0, 1835.0,

In [19]:
## Use my function to build the data frame

df_year = pd.DataFrame(columns=['year','total_books', 'total_pages', 'most_prolific_month', 'longest_book'])
for i in set(years):
    year = int(i)
    total_books, total_pages, most_prolific_month, longest_book = year_historic(int(i))
    df_year = df_year.append({'year': int(i), 'total_books': total_books , 'total_pages': total_pages, 'most_prolific_month': most_prolific_month, 'longest_book': longest_book}, ignore_index=True)
print(df_year.head(10))
print(df_year.tail(10))

   year total_books  total_pages most_prolific_month  \
0  1678           1        257.0                   1   
1  1679           2        958.0                   1   
2  1680           1        182.0                   1   
3  1681           1        297.0                   1   
4  1682           2        132.0                   1   
5  1684           3        658.0                   1   
6  1686           4       1472.0                   1   
7  1688           2        112.0                   1   
8  1689           1         63.0                   1   
9  1691           2        429.0                   1   

                                        longest_book  
0                                 title publicati...  
1               title publication_date  num_pages...  
2                                                ...  
3                 title publication_date  num_pag...  
4                                                ...  
5                                                ... 

> ## RQ3- Report2
>
> the following is LLM code suggested as a solution:

In [ ]:
def year_historic(year):
    # Filter the data for the given year
    df_year = df[df['year'] == year]

    # The number of books published that year
    total_books = df_year['num_books'].sum()

    # The total number of pages written that year
    total_pages = df_year['num_pages'].sum()

    # The most prolific month of that year
    most_prolific_month = df_year['month'].mode()[0]

    # The longest book written that year
    longest_book = df_year[df_year['num_pages'] == df_year['num_pages'].max()]['longest book'].values[0]

    return total_books, total_pages, most_prolific_month, longest_book

There are many problems with this solution and clearly its not a correct one.
1. Columns- we dont have 'year', 'num_books' and 'month' columns in our datset and we need to creat them.
2. Types- the data type of 'num_pages' is not int and has many values that needs to be ignored.
   also to extract 'year' there are many operation needed in order to 'clean' the data.
3. Running time- with this approach we'll hade very long running time to construct the full data frame,  
   because we have to run over the whole books dataset for each year.